# 학습목표
- 시계열을 학습 데이터로 변환
- LSTM으로 악보 예측

In [1]:
# 코드 기준 번호 정의 (인코딩)
code2idx = {'c4':0, 'd4':1, 'e4':2, 'f4':3, 'g4':4, 'a4':5, 'b4':6,
            'c8':7, 'd8':8, 'e8':9, 'f8':10, 'g8':11, 'a8':12, 'b8':13}

# 번호 기준 코드 정의 (디코딩)
idx2code = {0:'c4', 1:'d4', 2:'e4', 3:'f4', 4:'g4', 5:'a4', 6:'b4',
            7:'c8', 8:'d8', 9:'e8', 10:'f8', 11:'g8', 12:'a8', 13:'b8'}

# 시퀀스 데이터 정의
seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8',
       'g8', 'g8', 'g4', 'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4',
       'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4', 'd8', 'd8', 'd8',
       'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8',
       'g4', 'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8',
       'g8', 'e8', 'e8', 'e4']

In [2]:
# 소리변환 함수
# o : 옥타브, s : 음표
def freq(o, s):
    if s[0] == 'c':     return 524*2**o
    elif s[0] == 'd':   return 587*2**o
    elif s[0] == 'e':   return 659*2**o
    elif s[0] == 'f':   return 698*2**o
    elif s[0] == 'g':   return 784*2**o
    elif s[0] == 'a':   return 880*2**o
    elif s[0] == 'b':   return 988*2**o

In [5]:
# 플레이어
from IPython.display import Audio, display
import numpy as np
import time

def single_tone(frequecy, s, sampling_rate=22050, duration=0.1):
    # frequency: 주파수
    # s : 음표 (4분음표, 8분음표)
    # sampling_rate: 초당 샘플링 데이터 수
    # duration: 지속 시간. 단위 초. 디폴트 1초
    t = np.linspace(0, duration, int(sampling_rate))
    y = np.sin(2 * np.pi * frequecy * t)
    time.sleep(s)
    return y

def play_sound(seq) :
  result = []

  for i in range(len(seq)) :
    # 딜레이를 조절하여 박자를 맞춤
    if seq[i][1] == "4" :
        result.append(single_tone(freq(2, seq[i]), 0.5))
    elif seq[i][1] == "8" :
        result.append(single_tone(freq(2, seq[i]), 0.25))

    octave = np.hstack(result)

  display(Audio(octave, rate=44100))

In [6]:
play_sound(seq)

Output hidden; open in https://colab.research.google.com to view.

- 시계열을 학습 데이터(특성, 라벨)로 변환하는 함수

In [8]:
# 노래 데이터를 window_size만큼의 크기로 각각 분리
def split_seq(seq,window_size):
    data_set=[]

    for i in range(len(seq)-window_size):

        # window_size+1만큼 잘라서 저장
        subset = seq[i:(i+window_size+1)]

        # 음표를 인코딩해서 저장
        data_set.append([code2idx[item] for item in subset])

    return data_set

In [ ]:
import numpy as np

data_set2 = np.array(split_seq(seq, 4))
print(data_set2.shape)
print(data_set2)

In [23]:
X=data_set2[:,:4]
y=data_set2[:,-1]
print(x.shape, y.shape)

(50, 4) (50,)


- 라벨을 원핫인코딩

In [24]:
from tensorflow.keras.utils import to_categorical

# 정규화 (0-1사이 값으로변환)
X=X/13.0
y=to_categorical(y)
print(y.shape)

(50, 12)


- 특성 데이터의 특성(시간) 차원을 추가
  - 온도 데이터 시계열, 습도 데이터 시계열, 기압 데이터 시계열 (특성차원) -> 날씨 예측
- f8 -> 1개의 특성
- f와 8을 분리하면 -> 2개의 특성

In [26]:
X_ex=np.reshape(X,(50,4,1))
print(X_ex.shape)

(50, 4, 1)


- 신경망 설계

In [28]:
# prompt: 신경망 설계

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
model1 = Sequential()

# (1,4,1) : 배치 크기, 특성 데이터의 시간 수, 특성 수(음계+박자)
# stateful=True : 1개씩 처리해서 다음 단계로 결과를 넘겨준다

model1.add(LSTM(units=128,batch_input_shape=(1,4,1),stateful=True))
model1.add(Dense(12, activation='softmax'))
model1.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (1, 128)                  66560     
                                                                 
 dense_1 (Dense)             (1, 12)                   1548      
                                                                 
Total params: 68108 (266.05 KB)
Trainable params: 68108 (266.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
# prompt: 컴파일

model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
for epoch_idx in range(500):

  # shuffle=False : 악보는 섞으면 안되므오 False로 설정
  # batch_size=1 : 악보라서 하나씩
  model1.fit(X_ex,y,epochs=1, batch_size=1,verbose=2,shuffle=False)

  # stateful 을 설정하고 리셋하지 않으면 이전과 값들이 더해서 쌓이므로 리셋 필요
  model1.reset_states()

50/50 - 2s - loss: 2.3987 - accuracy: 0.1600 - 2s/epoch - 49ms/step
50/50 - 0s - loss: 2.0208 - accuracy: 0.3400 - 145ms/epoch - 3ms/step
50/50 - 0s - loss: 1.9632 - accuracy: 0.3400 - 144ms/epoch - 3ms/step
50/50 - 0s - loss: 1.9462 - accuracy: 0.3400 - 146ms/epoch - 3ms/step
50/50 - 0s - loss: 1.9366 - accuracy: 0.3400 - 141ms/epoch - 3ms/step
50/50 - 0s - loss: 1.9297 - accuracy: 0.3400 - 143ms/epoch - 3ms/step
50/50 - 0s - loss: 1.9243 - accuracy: 0.3400 - 150ms/epoch - 3ms/step
50/50 - 0s - loss: 1.9197 - accuracy: 0.3400 - 203ms/epoch - 4ms/step
50/50 - 0s - loss: 1.9157 - accuracy: 0.3400 - 205ms/epoch - 4ms/step
50/50 - 0s - loss: 1.9120 - accuracy: 0.3400 - 394ms/epoch - 8ms/step
50/50 - 0s - loss: 1.9084 - accuracy: 0.3600 - 222ms/epoch - 4ms/step
50/50 - 0s - loss: 1.9046 - accuracy: 0.3600 - 203ms/epoch - 4ms/step
50/50 - 0s - loss: 1.9004 - accuracy: 0.3600 - 206ms/epoch - 4ms/step
50/50 - 0s - loss: 1.8955 - accuracy: 0.3600 - 285ms/epoch - 6ms/step
50/50 - 0s - loss: 1.8

- 악보예측

In [34]:
co = 50 # 예측할 갯수

# 초기악보
seq_out = ['d8','e8','f4','f8']

# 초기 악보 4개로 전체 악보 예측
pred_out = model1.predict(X_ex, batch_size=1)

50/50 [==============================] - 0s 2ms/step


In [35]:
# 50개 예측 악보 출력
for  i in range(co) :

  # 가장 큰 확률 값을 가진 악보의 인덱스를 반환
  pred_i = np.argmax(pred_out[i])

  # 해당 인덱스의 악보를 기존 4개의 악보에 이어서 저장
  seq_out.append(idx2code[pred_i])

print("원래악보",seq)
print("예측악보",seq_out)

원래악보 ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4', 'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4', 'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4', 'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']
예측악보 ['d8', 'e8', 'f4', 'f8', 'd8', 'd8', 'd8', 'd8', 'g8', 'd8', 'e8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'e8', 'f4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e8', 'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'd8', 'e8', 'e8', 'e8', 'e8', 'e8', 'f4', 'e8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']


In [33]:
play_sound(seq)

Output hidden; open in https://colab.research.google.com to view.